In [15]:
import pandas as pd
import requests
import json
url = "https://api.cartola.globo.com/atletas/mercado"
response = requests.get(url)
json_data = response.json()

In [16]:
data_dict = json.loads(response.text)

In [17]:
atletas_df = pd.json_normalize(data_dict["atletas"])

In [18]:
clubes_df = pd.DataFrame.from_dict(data_dict["clubes"], orient='index')
clubes_df.rename(columns={'id': 'clube_id'}, inplace=True)

In [19]:
posicoes_df = pd.DataFrame.from_dict(data_dict["posicoes"], orient='index')
posicoes_df.rename(columns={'id': 'posicao_id'}, inplace=True)

In [20]:
status_df = pd.DataFrame.from_dict(data_dict["status"], orient='index')
status_df.rename(columns={'id': 'status_id'}, inplace=True)

In [21]:
rodada_atual = atletas_df.loc[0,'rodada_id']
partidas_response = requests.get('https://api.cartola.globo.com/partidas/'+str(rodada_atual))
partidas_json_data = partidas_response.json()
partidas_df = pd.json_normalize(partidas_json_data["partidas"])

In [25]:
# Adicionar uma nova coluna 'é_mandante' no clubes_df
clubes_df['mandante'] = clubes_df['clube_id'].isin(partidas_df['clube_casa_id'])

# Função para determinar se a posição do clube é maior ou menor que a do adversário
def comparar_posicao(clube_id, partidas_df):
    for _, row in partidas_df.iterrows():
        if row['clube_casa_id'] == clube_id:
            return 'maior' if row['clube_casa_posicao'] > row['clube_visitante_posicao'] else 'menor'
        elif row['clube_visitante_id'] == clube_id:
            return 'maior' if row['clube_visitante_posicao'] > row['clube_casa_posicao'] else 'menor'
    return 'N/A'

# Adicionar a nova coluna no clubes_df
clubes_df['posicao_comparada'] = clubes_df['clube_id'].apply(lambda x: comparar_posicao(x, partidas_df))
